In [1]:
import pandas as pd
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')
from gensim import corpora, models

[nltk_data] Downloading package wordnet to /Users/xijiahu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
news = pd.read_csv('FinalNews.csv', error_bad_lines=False)

In [3]:
def lemmatize_stemming(text):
    return WordNetLemmatizer().lemmatize(text, pos='v')

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [4]:
processed_docs = news['Ncontent'].map(preprocess)

In [5]:
dictionary = gensim.corpora.Dictionary(processed_docs)

In [6]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 ability
1 accord
2 accuse
3 acknowledge
4 active
5 addition
6 advertise
7 advertisement
8 advocacy
9 affair
10 affiliate


In [7]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [8]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)

In [9]:
from pprint import pprint
corpus_tfidf = tfidf[bow_corpus]
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.031525414207240324),
 (1, 0.03991540284444522),
 (2, 0.036970420646929285),
 (3, 0.03062440756267819),
 (4, 0.02937971222753259),
 (5, 0.02937971222753259),
 (6, 0.016132867695373043),
 (7, 0.02898984994787927),
 (8, 0.06039238321867404),
 (9, 0.045896513380327866),
 (10, 0.03804194767745612),
 (11, 0.018658185256284442),
 (12, 0.021791800991428693),
 (13, 0.05921953857646538),
 (14, 0.03249322108187023),
 (15, 0.03249322108187023),
 (16, 0.013190072598799065),
 (17, 0.04952275929892806),
 (18, 0.03062440756267819),
 (19, 0.07608389535491224),
 (20, 0.02978157213458381),
 (21, 0.05184596383269263),
 (22, 0.018833482069544907),
 (23, 0.04145894881001391),
 (24, 0.02476137964946403),
 (25, 0.035919545287343764),
 (26, 0.013421218748086623),
 (27, 0.02157230983310134),
 (28, 0.02395332298095002),
 (29, 0.021014188342539455),
 (30, 0.1146573293566666),
 (31, 0.02937971222753259),
 (32, 0.025479406523703688),
 (33, 0.03467484995219816),
 (34, 0.07676518674101063),
 (35, 0.03804194767

In [10]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=5, id2word=dictionary, passes=2, workers=2)

In [11]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.007*"claim" + 0.007*"tell" + 0.006*"investigation" + 0.006*"kogan" + 0.006*"wylie" + 0.006*"harvest" + 0.006*"leave" + 0.005*"know" + 0.005*"share" + 0.005*"personal"
Topic: 1 
Words: 0.008*"like" + 0.008*"congress" + 0.008*"zuckerberg" + 0.008*"target" + 0.007*"india" + 0.006*"know" + 0.006*"party" + 0.006*"committee" + 0.006*"profile" + 0.005*"report"
Topic: 2 
Words: 0.010*"profile" + 0.009*"employee" + 0.008*"voter" + 0.007*"wylie" + 0.007*"parscale" + 0.007*"analyticas" + 0.007*"mercer" + 0.006*"business" + 0.006*"effort" + 0.006*"know"
Topic: 3 
Words: 0.011*"report" + 0.009*"profile" + 0.009*"group" + 0.007*"voter" + 0.006*"investigation" + 0.006*"zuckerberg" + 0.006*"privacy" + 0.005*"analyticas" + 0.005*"mercer" + 0.005*"party"
Topic: 4 
Words: 0.009*"obtain" + 0.009*"personal" + 0.007*"share" + 0.006*"kogan" + 0.006*"voter" + 0.006*"british" + 0.006*"party" + 0.006*"access" + 0.006*"help" + 0.006*"executive"


In [12]:
scoresperdoc=lda_model.inference(bow_corpus)
with open('Newstopic_news.tsv', "w",encoding="utf-8") as fo:
    for row in scoresperdoc[0]:
       fo.write("\t".join(["{:0.3f}".format(score) for score in row]))
       fo.write("\n")

In [13]:
release = pd.read_csv('fb_release1116.csv')

In [14]:
def lemmatize_stemming(text):
    return WordNetLemmatizer().lemmatize(text, pos='v')

def preprocess(text):
    result1 = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result1.append(lemmatize_stemming(token))
    return result1

In [15]:
processed_docs1 = release['content'].map(preprocess)

In [16]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
bow_corpus1 = [dictionary.doc2bow(doc) for doc in processed_docs1]

In [17]:
doc_lda = lda_model[bow_corpus1]

In [18]:
scoresperdoc1=lda_model.inference(bow_corpus1)
with open('Newstopic_release.tsv', "w",encoding="utf-8") as fo:
    for row in scoresperdoc1[0]:
       fo.write("\t".join(["{:0.3f}".format(score) for score in row]))
       fo.write("\n")

In [19]:
topic_news = pd.read_csv("Newstopic_news.tsv", sep='\t', names = ["topic0", "topic1", "topic2", "topic3", 'topic4'])

In [20]:
topic_release = pd.read_csv("Newstopic_release.tsv", sep='\t', names = ["topic0", "topic1", "topic2", "topic3", 'topic4'])

In [21]:
news_topic = topic_news.join(news)

In [22]:
news_topic

,topic0,topic1,topic2,topic3,topic4,Index,date,Ntitle,Ncontent
0,0.204,0.204,433.184,0.204,0.204,0,"June 15, 2018","['excambridge', 'analytica', 'staff', 'working...","['highlight', 'ap', 'confirmed', 'least', 'fou..."
1,0.204,0.204,435.183,0.205,0.204,1,"June 15, 2018","['excambridge', 'analytica', 'staff', 'working...","['highlight', 'ap', 'confirmed', 'least', 'fou..."
2,0.204,0.204,373.184,0.204,0.204,2,"June 18, 2018","['trump', '2020', 'working', 'excambridge', 'a...","['abstract', 'mbowers', 'ap', 'learned', 'dona..."
3,0.204,0.204,369.184,0.204,0.204,3,"June 17, 2018","['trump', '2020', 'working', 'excambridge', 'a...","['highlight', 'mbowers', 'ap', 'learned', 'don..."
4,0.202,0.204,0.203,13.189,0.202,4,"March 30, 2018","['letter', 'could', 'cambridge', 'analytica', ...","['highlight', 'instead', 'influencing', 'elect..."
5,0.202,0.204,0.203,13.189,0.202,5,"March 30, 2018","['letter', 'could', 'cambridge', 'analytica', ...","['highlight', 'instead', 'influencing', 'elect..."
6,0.204,0.204,409.184,0.204,0.204,6,"June 17, 2018","['revealed', 'firm', 'run', 'former', 'cambrid...","['·', 'excambridge', 'analytica', 'staffer', '..."
7,8.897,0.205,1.490,151.837,88.571,7,"May 15, 2018","['cambridge', 'analytica', 'parent', 'company'...","['dateline', 'london', 'abstract', 'corporate'..."
8,250.962,54.886,0.205,0.205,37.743,8,"March 22, 2018","['facebook', 'ceo', 'mark', 'zuckerberg', 'cam...","['1136', 'facebook', 'ceo', 'mark', 'zuckerber..."
9,0.205,110.181,0.204,0.206,0.205,9,"March 24, 2018","['cambridge', 'analytica', 'scandal', 'governm...","['368', 'cambridge', 'analytica', 'scandal', '..."


In [23]:
news_topic = news_topic.rename(index=str, columns={"date.1": "news_date"})

In [24]:
news_topic['date'] = pd.to_datetime(news_topic['date'].replace(',',''), format = ' %B %d, %Y')

In [25]:
news_topic.to_csv('news_topic.csv', index_label='Index')

In [26]:
release_topic = topic_release.join(release)

In [27]:
release_topic

,topic0,topic1,topic2,topic3,topic4,Index_name,date,title,content
0,5.379,120.712,5.498,0.205,0.206,0,"October 8, 2018",Introducing Portal From Facebook: New Video Ca...,"['could', 'easily', 'connect', 'closest', 'fri..."
1,0.206,0.207,0.205,0.205,22.177,1,"September 26, 2018",Introducing Oculus Quest _� a New All-in-One V...,"['today', 'oculus', 'connect', '5', 'oc5', 'in..."
2,23.440,27.520,26.630,0.205,0.205,2,"September 23, 2018",Introducing the Facebook Community Leadership ...,"['ime', 'archibong', 'vice', 'president', 'pro..."
3,23.688,23.421,69.480,0.206,0.205,3,"February 9, 2018",A New Investment in Community Leaders,"['jennifer', 'dulski', 'head', 'group', 'commu..."
4,0.206,112.717,0.219,0.205,25.653,4,"February 6, 2018",Safer Internet Day: Teaching Children to Safel...,"['antigone', 'davis', 'global', 'head', 'safet..."
5,5.895,0.207,0.206,0.206,31.487,5,"February 4, 2018","This Friends Day, Show Gratitude for Your Clos...","['mike', 'nowak', 'product', 'director', 'good..."
6,0.204,0.205,0.208,6.179,0.204,6,"January 31, 2018",Facebook Reports Fourth Quarter and Full Year ...,"['clickherefor', 'detail', 'facebook�s', 'fina..."
7,16.914,102.603,16.072,0.205,0.206,7,"January 29, 2018",More Local News on Facebook,"['alex', 'hardiman', 'head', 'news', 'product'..."
8,0.274,224.771,0.206,15.543,0.206,8,"January 28, 2018",Giving You More Control of Your Privacy on Fac...,"['erin', 'egan', 'chief', 'privacy', 'officera..."
9,214.955,299.341,35.537,2.961,0.207,9,"January 25, 2018",Guest Post: Is Social Media Good or Bad For De...,"['toomas', 'hendrik', 'ilves', 'distinguished'..."


In [28]:
release_topic['date'] = pd.to_datetime(release_topic['date'].replace(',',''), format = '%B %d, %Y')

In [29]:
release_topic.to_csv('release_topic.csv', index_label= 'Index')

In [30]:
new_topic = news_topic.drop([ 'Ntitle','Ncontent'], axis=1)

In [31]:
new_topic.to_csv('news_topic.csv')

In [ ]:
release_topic2 = pd.read_csv('release_topic.csv')

In [32]:
new_topic = new_topic.rename(index=str, columns={"topic0": "n_topic0", "topic1": "n_topic1", "topic2": "n_topic2", "topic3": "n_topic3", "topic4": "n_topic4"})

In [33]:
result = new_topic.merge(release_topic, left_on='date', right_on='date', how='outer')

In [34]:
result = result.drop([ 'title','content','Index_name','Index'], axis=1)

In [35]:
result.to_csv('result1222.csv')

In [36]:
result

,n_topic0,n_topic1,n_topic2,n_topic3,n_topic4,date,topic0,topic1,topic2,topic3,topic4
0,0.204,0.204,433.184,0.204,0.204,2018-06-15,NaN,NaN,NaN,NaN,NaN
1,0.204,0.204,435.183,0.205,0.204,2018-06-15,NaN,NaN,NaN,NaN,NaN
2,0.204,0.204,373.184,0.204,0.204,2018-06-18,NaN,NaN,NaN,NaN,NaN
3,0.204,0.204,235.184,0.204,0.204,2018-06-18,NaN,NaN,NaN,NaN,NaN
4,0.204,0.204,369.184,0.204,0.204,2018-06-17,NaN,NaN,NaN,NaN,NaN
5,0.204,0.204,409.184,0.204,0.204,2018-06-17,NaN,NaN,NaN,NaN,NaN
6,0.202,0.204,0.203,13.189,0.202,2018-03-30,NaN,NaN,NaN,NaN,NaN
7,0.202,0.204,0.203,13.189,0.202,2018-03-30,NaN,NaN,NaN,NaN,NaN
8,0.205,14.447,0.206,7.506,8.636,2018-03-30,NaN,NaN,NaN,NaN,NaN
9,0.205,0.204,28.181,0.206,0.205,2018-03-30,NaN,NaN,NaN,NaN,NaN


In [37]:
result1 = result.set_index('date').groupby(pd.Grouper(freq='D')).mean()

In [38]:
result1

,n_topic0,n_topic1,n_topic2,n_topic3,n_topic4,topic0,topic1,topic2,topic3,topic4
date,,,,,,,,,,
2017-03-04,156.45700,0.20500,8.25200,43.880,0.2050,NaN,NaN,NaN,NaN,NaN
2017-03-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-03-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-03-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-03-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-03-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-03-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-03-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-03-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
result1.to_csv('result1222D.csv')

In [215]:
result1.to_csv('result1210.csv')

In [40]:
result2 = result.set_index('date').groupby(pd.Grouper(freq='W')).mean()

In [41]:
result2.to_csv('result1222w.csv')